In [2]:
%load_ext autoreload
%autoreload 2

##################################
##################################
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Imports
from pynattas.builder.netBuilder import GenericNetwork 
from pynattas.utils import layerCoder

import configparser
import pynattas as pnas
from datetime import datetime
import time

config = configparser.ConfigParser()
config.read('config.ini')


nas_check = config.getboolean(section='Mode', option='network_architecture_search')
ht_check = config.getboolean(section='Mode', option='hyperparameter_tuning')


task = config['Mode']['task']
if task not in ['C','D']:
    print(f"Error: Selected Task {task} is not available. Check config_optimizer.ini for verification.")
    exit()
else:
    print(f"Task: {task}")
# Define NAS parameters and run GA
max_layers = config.getint('GeneralNetwork', 'max_layers')
max_iterations = int(config['GA']['max_iterations'])
population_size = int(config['GA']['population_size'])
log_path = str(config['GA']['logs_dir_GA'])
mating_pool_cutoff = float(config['GA']['mating_pool_cutoff'])
mutation_probability = float(config['GA']['mutation_probability'])
############################################
print(f"Max Layers: {max_layers}")
print(f"Max Iterations: {max_iterations}")
print(f"Population Size: {population_size}")
print(f"Log Path: {log_path}")
print(f"Mating Pool Cutoff: {mating_pool_cutoff}")
print(f"Mutation Probability: {mutation_probability}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Task: C
Max Layers: 12
Max Iterations: 4
Population Size: 8
Log Path: ./logs/GA_logs
Mating Pool Cutoff: 0.8
Mutation Probability: 0.1


In [ ]:
# Define population
from pynattas.builder.population import Population
import torch 

pop = Population(max_layers=max_layers, n_individuals=population_size)
print(f"Initial Population: {len(pop.population)}")
print(f"Current Generation: {pop.iteration}")

idx = 0
input_channels = 3
input_height = 224
input_width = 224
num_classes = 2

for individual in pop.population:
    try:
        Net = GenericNetwork(
                individual.architecture, 
                input_channels,
                input_height,
                input_width,
                num_classes,)

        Net.build()

        x = torch.randn(1, 3, 224, 224)
        Net(x)
        print(Net.get_param_size())
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Reset memory
        torch.cuda.empty_cache()
        del Net

Initial Population: 8
Current Generation: 0


Building layer 14/17: AvgPool:  82%|████████▏ | 14/17 [00:00<00:00, 238.77layer/s]       


Error: Kernel size is too large for the current dimensions: Heigt=(3), Width=(3)


Building layer 15/17: ConvSE:  88%|████████▊ | 15/17 [00:00<00:00, 154.41layer/s]        


Error: Kernel size 5 is too large for the current dimensions: Heigt=(3), Width=(3)


...

Building layer 17/21: ConvBnAct:  81%|████████  | 17/21 [00:00<00:00, 465.45layer/s]     

Error: Kernel size 5 is too large for the current dimensions: Heigt=(3), Width=(3)



Building layer 13/13: ClassificationHead: 100%|██████████| 13/13 [00:00<00:00, 34.37layer/s]


361.067


Building layer 17/17: ClassificationHead: 100%|██████████| 17/17 [00:09<00:00,  1.83layer/s]


In [ ]:
Net = GenericNetwork(
        individual.architecture, 
        input_channels,
        input_height,
        input_width,
        num_classes,)

Net.build()


x = torch.randn(1, 3, 224, 224)
Net(x)
print(Net.get_param_size())

In [6]:
# from pynattas.optimizers import ga

In [ ]:
# nas_result = pnas.optimizers.ga.ga_optimizer(
#     max_layers=max_layers,
#     max_iter=max_iterations,
#     n_individuals=population_size,
#     mating_pool_cutoff=mating_pool_cutoff,
#     mutation_probability=mutation_probability,
#     logs_directory=log_path,
#     task=task,
# )